## Julia ODE Benchmark

1. I am going to use the example from the discussion in the performance location of the Julia ODE write up.  
    1. I am going to use the speed up tricks there.
    1. I am going to compare to Python with what seems to be the default set up in Python
1. To be fair I am going to implement the Python example directly in Julia.
1. This seems fair
    1. Implement the Julia example in Python 
    1. Implement the Python example in Julia
    
They can not be in the same notebook.  I do not know how to have more than one kernel per notebook!

In [31]:
using StaticArrays
function lorenz(u, p, t)
    dx = 10.0 * (u[2] - u[1])
    dy = u[1] * (28.0 - u[3]) - u[2]
    dz = u[1] * u[2] - (8 / 3) * u[3]
    [dx, dy, dz]
end
function lorenz!(du, u, p, t)
    du[1] = 10.0 * (u[2] - u[1])
    du[2] = u[1] * (28.0 - u[3]) - u[2]
    du[3] = u[1] * u[2] - (8 / 3) * u[3]
    nothing
end
function lorenz_static(u, p, t)
    dx = 10.0 * (u[2] - u[1])
    dy = u[1] * (28.0 - u[3]) - u[2]
    dz = u[1] * u[2] - (8 / 3) * u[3]
    SA[dx, dy, dz]
end

using Suppressor 
@suppress_err using DifferentialEquations, BenchmarkTools
u0 = [1.0; 0.0; 0.0]; u0SA =SA[1.0; 0.0; 0.0]
tspan = (0.0, 100.0)
prob = ODEProblem(lorenz, u0, tspan)
prob! = ODEProblem(lorenz!, u0, tspan, inplace=true)
probStatic = ODEProblem(lorenz_static, u0, tspan, inplace=true)
probStaticStatic = ODEProblem(lorenz_static, u0SA, tspan, inplace=true)
probStaticStaticOnlyEnd = ODEProblem(lorenz_static, u0SA, tspan,inplace=true,save_everystep = false);

In [33]:
alg=Tsit5()
@suppress_err ( @btime solve(prob, alg))
@suppress_err ( @btime solve(prob!, alg))
@suppress_err ( @btime solve(probStatic, alg))
@suppress_err ( @btime solve(probStaticStatic, alg))
@suppress_err ( @btime solve(probStaticStaticOnlyEnd, alg));

  4.517 ms (100176 allocations: 10.81 MiB)
  930.200 μs (11771 allocations: 1.37 MiB)
  1.619 ms (34040 allocations: 3.75 MiB)
  467.500 μs (1361 allocations: 454.06 KiB)
  281.100 μs (47 allocations: 5.52 KiB)


Julia will pick an algorithm if we do not tell it what algorithm to use. It may be faster.  It may not!

In [32]:
@suppress_err ( @btime solve(prob))
@suppress_err ( @btime solve(prob!))
@suppress_err ( @btime solve(probStatic))
@suppress_err ( @btime solve(probStaticStatic))
@suppress_err ( @btime solve(probStaticStaticOnlyEnd));

  4.277 ms (102806 allocations: 11.12 MiB)
  781.200 μs (11819 allocations: 1.40 MiB)
  1.578 ms (34065 allocations: 3.78 MiB)
  424.600 μs (1383 allocations: 486.95 KiB)
  256.600 μs (61 allocations: 6.09 KiB)


We are going to understand what "stiff" means. But first we need an example. 

In [34]:
function rober!(du, u, p, t)
    y1, y2, y3 = u
    k1, k2, k3 = p
    du[1] = -k1 * y1 + k3 * y2 * y3
    du[2] = k1 * y1 - k2 * y2^2 - k3 * y2 * y3
    du[3] = k2 * y2^2
    nothing
end
prob = ODEProblem(rober!, [1.0, 0.0, 0.0], (0.0, 1e6), [0.04, 3e7, 1e4])
sol = solve(prob,Tsit5())

┌ Warning: Interrupted. Larger maxiters is needed. If you are using an integrator for non-stiff ODEs or an automatic switching algorithm (the default), you may want to consider using a method for stiff equations. See the solver pages for more details (e.g. https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/#Stiff-Problems).
└ @ SciMLBase C:\Users\struther\.julia\packages\SciMLBase\QqtZA\src\integrator_interface.jl:504


retcode: MaxIters
Interpolation: specialized 4th order "free" interpolation
t: 999997-element Vector{Float64}:
   0.0
   0.0014148468219250373
   0.0020449182545311173
   0.0031082402716566307
   0.004077787050059496
   0.005515332443361059
   0.007190040962774541
   0.009125372578778032
   0.011053912492732977
   0.012779077276958607
   0.014024022307121159
   0.015243850666599433
   0.01653506083093412
   ⋮
 627.2190396080226
 627.2195546456298
 627.2200696831143
 627.2205847204418
 627.2210997576807
 627.221614794865
 627.2221298319947
 627.2226448690358
 627.2231599060223
 627.2236749429201
 627.2241899796951
 627.2247050163814
u: 999997-element Vector{Vector{Float64}}:
 [1.0, 0.0, 0.0]
 [0.9999434113193613, 3.283958829839966e-5, 2.374909234028646e-5]
 [0.9999182177783585, 3.5542680136344576e-5, 4.6239541505020636e-5]
 [0.999875715036629, 3.630246933484973e-5, 8.798249403609502e-5]
 [0.9998369766077329, 3.646280308115454e-5, 0.00012656058918590176]
 [0.9997795672444667, 3.646643085

In [35]:
@btime sol = solve(prob)

  128.800 μs (1505 allocations: 122.94 KiB)


retcode: Success
Interpolation: specialized 4th order "free" interpolation, specialized 2nd order "free" stiffness-aware interpolation
t: 86-element Vector{Float64}:
      0.0
      0.0014148468219250373
      0.0020449182545311173
      0.0031082402716566307
      0.004077787050059496
      0.005515332443361059
      0.007190040962774541
      0.009125372578778032
      0.011053912492732977
      0.012779077276958607
      0.014024022307121159
      0.015243850666599433
      0.01653506083093412
      ⋮
 134150.3739391502
 159907.25511463825
 190651.00555599324
 227529.00448704563
 272100.45016950485
 326618.040391603
 394675.44922522793
 483135.46322738164
 611978.440308988
 832001.9858022375
 955418.4664430803
      1.0e6
u: 86-element Vector{Vector{Float64}}:
 [1.0, 0.0, 0.0]
 [0.9999434113193613, 3.283958829839966e-5, 2.374909234028646e-5]
 [0.9999182177783585, 3.5542680136344576e-5, 4.6239541505020636e-5]
 [0.999875715036629, 3.630246933484973e-5, 8.798249403609502e-5]
 [0.999836

# Stiff means twitchy